In [1]:
import itertools
import pickle
import random

import numpy as np
import pandas as pd
import torch
from transformers import BertForNextSentencePrediction, BertTokenizer

from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
pseudowords = []
for i in range(15):
    pseudowords.append(np.load(f"../../data/pseudowords/bert/pseudowords_comapp_bert_{i*37}_{i*37+37}.npy"))
pseudowords = np.concatenate(pseudowords)

csv_data = []
for i in range(1, 16):
    csv_data.append(pd.read_csv(f"../../data/pseudowords/bert/order_bert_{i}.csv", sep=";", index_col=0, header=None, quotechar="|", names=["order", "label"]))
csv_data = pd.concat(csv_data)

bert_tokens = [d[0] for d in csv_data.values]
bert_tokens

['""Was13',
 '"647',
 '"Wir-äh-spielen-äh-in-der-äh-Champions-League647',
 '(1597',
 '(1600',
 '(1602',
 '(1624',
 '(1637',
 '(1639',
 '(1641',
 '(1643',
 '(1645',
 '(379',
 '(579',
 '(581',
 '(584',
 '(590',
 '(592',
 '(600',
 '(886',
 '(889',
 '(892',
 '(900',
 '(905',
 '(907',
 '(909',
 '(911',
 '(917',
 '(919',
 '(921',
 '(923',
 ')1597',
 ')1600',
 ')1602',
 ')1624',
 ')1637',
 ')1639',
 ')1641',
 ')1643',
 ')1645',
 ')1792',
 ')379',
 ')579',
 ')581',
 ')584',
 ')590',
 ')592',
 ')600',
 ')886',
 ')889',
 ')892',
 ')900',
 ')905',
 ')907',
 ')909',
 ')911',
 ')917',
 ')919',
 ')921',
 ')923',
 ')«579',
 ',1459',
 ',973',
 '-128',
 '-651',
 '-654',
 '-875',
 '-973',
 ':595',
 ':875',
 ':973',
 'Abstand683',
 'Allein20',
 'Aller1630',
 'Als1315',
 'Als133',
 'Als1770',
 'Am488',
 'Am492',
 'Am500',
 'Amerika605',
 'Anstatt320',
 'Art129',
 'Arzt1509',
 'Augenblick1301',
 'Ausmaß1777',
 'BRUTAL1503',
 'Besser1762',
 'Bis559',
 'Brutal1503',
 'Buche1346',
 'Das1313',
 'Das1461',
 'Da

In [3]:
model = BertForNextSentencePrediction.from_pretrained("bert-base-german-cased", return_dict=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')

combined_embeddings = torch.cat((model.bert.embeddings.word_embeddings.weight, torch.tensor(pseudowords)), dim=0)
model.bert.embeddings.word_embeddings = torch.nn.Embedding.from_pretrained(combined_embeddings)
tokenizer.add_tokens(bert_tokens)
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 30555. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(30555, 768)

In [4]:
with open("../../out/definitions.pickle", "rb") as file:
    definitions = pickle.load(file)
with open("../../out/sentences.pickle", "rb") as file:
    sentences = pickle.load(file)

In [7]:
def find_examples(definition, examples):
    predictions = {}
    for num, example in enumerate(examples):
        len_prompt = len(definition) + len(" For example: ") + len(example)
        if len_prompt > 512:
            prompt = definition[:512+len(definition)-len_prompt-1] + "… For example: "
        else:
            prompt = definition + " For example: "
        
        inputs = tokenizer(prompt, example, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        predictions[num] = logits[0, 0]  # probability that the next sentence makes sense
    res = max(predictions, key=predictions.get)
    return examples[res]

In [ ]:
random.seed(15)
attempts = 10
result = []
for false_positives in range(2, 8):
    for key, definition in tqdm(definitions.items()):
        for attempt in range(attempts):
            try:
                sentence = random.choice(list(sentences[int(key)]))
            except KeyError:
                print(None, None, None)
                result.append(pd.Series({"constr": key, "definition": definition, "example": None, "prediction": None, "correct": None}))
                continue
            others = itertools.chain.from_iterable([sentence_list for constr, sentence_list in sentences.items() if int(constr) != int(key)])
            others = random.choices(list(others), k=false_positives)
            examples = list(set(others) | {sentence})
            prediction = find_examples(definition, examples)
            print(prediction == sentence, sentence, prediction)
            result.append(pd.Series({"constr": key, "definition": definition, "example": sentence, "prediction": prediction, "correct": prediction == sentence}))
    result = pd.DataFrame(result)
    result.to_csv(f"../../out/comapp/result_1_vs_{false_positives}_{attempts}attempts_bert.tsv", sep="\t")

  0%|          | 0/211 [00:00<?, ?it/s]

True Da musst du dann die juristischen Begriffe benutzen , du musst dich damit abfinden , dass dein Mandant nicht versteht , worum es geht , geschweige denn das Publikum . Da musst du dann die juristischen Begriffe benutzen , du musst dich damit abfinden , dass dein Mandant nicht versteht , worum es geht , geschweige denn das Publikum .
False Bei ihnen ist es bereits schwer vorstellbar , dass sie Geiseln nehmen , geschweige denn diese ermorden , und schon gar keine Frauen oder sogar Kinder ."" Die südkoreanische Schriftstellerin Han Kang hat mit ihrem RomanThe Vegetarian ( " " Die Vegetarierin " " ) den diesjährigen Man Booker International Prize gewonnen .
True Dies lässt sich damit begründen , dass vor und neben Alexander – soweit wir wissen – kein Grieche auf die Idee gekommen ist , geschweige denn versucht hat , nach Indien zu ziehen . Dies lässt sich damit begründen , dass vor und neben Alexander – soweit wir wissen – kein Grieche auf die Idee gekommen ist , geschweige denn versuc

In [ ]:
# TODO Mit Pseudowords testen!